# Esercizio Lab-4 : Text segmentation

### Librerie principali

In [ ]:
import spacy
import numpy as np

### Elaborazione del testo

Nella funzione "text_processing" è stata usata la libreria di spacy per elaborare il documento: tokenizzazione e rimozione stopwords.

In [ ]:
def text_processing(filename):
    texts =[]
    spa = spacy.load('en_core_web_md')

    with open(filename, 'r', encoding="utf8") as f:
        texts = f.read().splitlines()
    sents = []
    for text in texts:
        doc = spa(text)
        for sent in doc.sents:
            sents.append(sent)

    tokens = [[token.lemma_.lower() for token in sent    
                        if not token.is_stop and 
                        not token.is_punct and token.text.strip() and 
                        len(token) >= 3] 
                        for sent in sents]
    tokenized_sents = [x for x in tokens if x] 

    return tokenized_sents, sents

### TextTiling

Per l'implentazione dell'algoritmo di TextTiling è stata usata la tecnica del vocabulary-management, come indicata nell'articolo di Hearst, che sfrutta il rapporto tra le nuove parole diviso per la lunghezza dell'intervallo.

In [ ]:
def vocabulary(sentences):
    new_words1 = set()
    new_words2 = set(sentences[0])
    scores=[]

    for i in range(1,len(sentences)-1):
        left_words = set(sentences[i-1]).difference(new_words1)
        right_words = set(sentences[i+1]).difference(new_words2)
        score = (len(left_words) + len(right_words)) / (len(sentences[i-1])+len(sentences[i+1]))
        scores.append(score)
        new_words1 = new_words1.union(sentences[i-1])
        new_words2 = new_words2.union(sentences[i+1])
    last = set(sentences[len(sentences)-1]).difference(new_words1)
    scores.append(len(last)/len(sentences[len(sentences)-1]))
    return scores

Le seguenti funzioni vengono utilizzate per decidere dove inserire i boundaries attraverso il calcolo dello score per individuare la posizione migliore.

In [ ]:
def get_boundaries(scores):
    boundaries=[]
    mean = np.mean(scores) -  np.std(scores)
    for i, score in enumerate(scores):
        depth_scores = depth_score(scores, i, "left") + depth_score(scores, i, "right")
        if depth_scores >= mean:
            boundaries.append(i)
    return boundaries

def depth_score(scores, current, side): #cerco gli score migliori su cui mettere un confine
    depth_score = 0
    i = current
    while scores[i] - scores[current] >= depth_score:
        depth_score = scores[i] - scores[current]

        if side == 'left':
            i = i - 1
            if (i < 0): break
        else:
            i = i + 1
            if (i == len(scores)): break
    return depth_score

### Conclusioni

Come possiamo vedere dal risultato, l'algortimo divide non solo il documento nei tre topic presenti, ma anche i paragrafi presenti in ognuno di essi con una buona accuratezza.

In [2]:
def print_results(boundaries, sentences):
    print(boundaries)
    for i, x in enumerate(sentences):
        print(sentences[i])
        if i in boundaries:
            print()

[2, 4, 6, 9, 14, 16, 23, 25, 47, 49, 51, 69, 75, 77, 82, 84, 86, 88, 89, 90, 91, 98]
Most work in discourse processing, both theoretical and computational, has focused on analysis of interclausal or intersentential phenomena.
This level of analysis is important for many discourse-processing tasks, such as anaphor resolution and dialogue generation.
However, important and interesting discourse phenomena also occur at the level of the paragraph.

This article describes a paragraph-level model of discourse structure based on the notion of subtopic shift, and an algorithm for subdividing expository texts into multi-paragraph "passages" or subtopic segments.
In this work, the structure of an expository text is characterized as a sequence of subtopical discussions that occur in the context of one or more main topic discussions.

Consider a 21-paragraph science news article, called Stargazers, whose main topic is the existence of life on earth and other planets.
Its contents can be described 